### Tips from prof

- Narrow scope of work (e.g. court level)

- Could try both binary/multi-class model outcomes and compare the performance 

- Change user from layperson to legal professional (and mention that this project is a stepping stone towards having layperson use the model)

- Link features to predicted outcome (if time permits can try using XGBoost with LIME for model interpretability)

- Can also try to see accuracy of models with different areas of law, lowest accuracy may be hardest area of law to predict


### Data setup

In [76]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import nltk
import pyLDAvis
import pyLDAvis.gensim_models

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [77]:
# Load CSV files into DataFrames
areas_of_law_df = pd.read_csv("data/prediction_data/areas_of_law.csv")
coram_df = pd.read_csv("data/prediction_data/coram.csv")
sg_legal_cases_df = pd.read_csv("data/prediction_data/sg_legal_cases_dataset.csv")
target_rulings_df = pd.read_csv("data/prediction_data/target_rulings.csv")
# Load the JSON file into a dictionary
with open('data/prediction_data/issues.json') as f:
    issues_data = [json.loads(line) for line in f]
issues_df = pd.DataFrame(issues_data)

# Load the JSON file into a dictionary
with open('data/rawish_data/facts.json') as f:
    facts_data = [json.loads(line) for line in f]
raw_facts_df = pd.DataFrame(facts_data)

# Merge DataFrames
merged_df = pd.merge(areas_of_law_df, coram_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, sg_legal_cases_df, left_on='casename', right_on='filename', how='outer')
merged_df = pd.merge(merged_df, issues_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, raw_facts_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, target_rulings_df, on='casename', how='outer')

merged_df.drop(columns=['Unnamed: 0'], inplace=True)
merged_df.drop(columns=['filename'], inplace=True)

# Display the resulting DataFrame
print(merged_df.head())

           casename                                        area_of_law  \
0   2000_SGCA_1.pdf  {'civil procedure': ['pleadings'], 'res judica...   
1  2000_SGCA_10.pdf  {'contract': ['formation'], 'equity': ['defenc...   
2  2000_SGCA_11.pdf  {'contract': ['discharge'], 'damages': ['asses...   
3  2000_SGCA_12.pdf  {'courts and jurisdiction': ['court of appeal'...   
4  2000_SGCA_13.pdf                     {'criminal law': ['offences']}   

                                         Coram court_level  \
0  ['Chao Hick Tin; L P Thean; Yong Pung How']        SGCA   
1   ['Chao Hick Tin; Tan Lee Meng; L P Thean']        SGCA   
2   ['Chao Hick Tin; Tan Lee Meng; L P Thean']        SGCA   
3   ['Chao Hick Tin; Tan Lee Meng; L P Thean']        SGCA   
4   ['Chao Hick Tin; Lai Kew Chai; L P Thean']        SGCA   

                                              issues  \
0  The claim was dismissed with costs by the\nHig...   
1  the claim and\nagainst that decision this appe...   
2  The appeal 

In [78]:
merged_df = merged_df.dropna()
merged_df.isna().sum()

casename       0
area_of_law    0
Coram          0
court_level    0
issues         0
facts          0
target         0
dtype: int64

### Data Preprocessing

In [79]:
nan_counts = merged_df.isna().sum()
print(nan_counts)

#nas are probably those reassigned cases, coram has 7, i just drop them for now
na_target_rows = merged_df[merged_df['target'].isna()]
print(na_target_rows)

merged_df.dropna(axis=0, inplace=True)
print(merged_df.isna().sum())

#remove empty lists
merged_df = merged_df.query("area_of_law != '[]'")

#target is unbalanced
target_counts = merged_df['target'].value_counts()
print(target_counts)

merged_df = merged_df.reset_index(drop=True) # prevent nan values from appearing after one-hot

casename       0
area_of_law    0
Coram          0
court_level    0
issues         0
facts          0
target         0
dtype: int64
Empty DataFrame
Columns: [casename, area_of_law, Coram, court_level, issues, facts, target]
Index: []
casename       0
area_of_law    0
Coram          0
court_level    0
issues         0
facts          0
target         0
dtype: int64
target
Favourable      3942
Unfavourable    2056
No outcome       795
Name: count, dtype: int64


In [80]:
merged_df['area_of_law'] = merged_df['area_of_law'].apply(ast.literal_eval)
merged_df['Coram'] = merged_df['Coram'].apply(ast.literal_eval)
merged_df.head(3)

,casename,area_of_law,Coram,court_level,issues,facts,target
0,2000_SGCA_1.pdf,"{'civil procedure': ['pleadings'], 'res judica...",[Chao Hick Tin; L P Thean; Yong Pung How],SGCA,The claim was dismissed with costs by the\nHig...,The facts\nThe appellant is the widow of one T...,Favourable
1,2000_SGCA_10.pdf,"{'contract': ['formation'], 'equity': ['defenc...",[Chao Hick Tin; Tan Lee Meng; L P Thean],SGCA,the claim and\nagainst that decision this appe...,facts and surrounding circumstances including ...,Favourable
2,2000_SGCA_11.pdf,"{'contract': ['discharge'], 'damages': ['asses...",[Chao Hick Tin; Tan Lee Meng; L P Thean],SGCA,The appeal \nThe questions which arise in this...,"Background \nThe first appellants, a French co...",No outcome


### Flatten areas_of_law

In [81]:
all_areas = []

for index, row in merged_df.iterrows():

    areas = row['area_of_law']
    flat_areas = []
    for main_area, sub_areas in areas.items():
        flat_areas.append(main_area)
        for sarea in sub_areas.copy():
            if len(sarea) > 33:
                sub_areas.remove(sarea)
        flat_areas.extend(sub_areas)
    all_areas.append(flat_areas)

In [82]:
for area in all_areas:
    if area == []:
        print(area)

### Topic Modelling

#### Helper functions for Topic Modelling 

In [83]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'\W*\b(?!no)\w{1,2}\b', '', text)
    stop_words = set(stopwords.words('english'))
    legal_stopwords = ('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'mrs', 'dr', 'mdm', 'court','version', 'hr', 'would', 'case', 'sghc', 'court', 'sgca', 'slr', 'sgdc', 'also', 'first', 'person', 'statement', 'line', 'para', 'fact', 'one', 'may', 'time', 'could', 'next', 'legal', 'issues', 'issue')
    stop_words.update(legal_stopwords)
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words

#### Topic Modelling For Facts

In [84]:
data = []

merged_df['processed_facts'] = merged_df['facts'].apply(preprocess_text)
merged_df.drop(columns=['facts'], inplace=True)
print(merged_df["processed_facts"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in merged_df['processed_facts']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# print(lda_model.show_topics())

# best_coherence = -1
# best_lda = None
# for num_topics in range(5, 31, 5):
#     # Train LDA model
#     lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                                 id2word=dictionary,
#                                                 num_topics=num_topics,
#                                                 random_state=42)
    
#     # Compute coherence score
#     coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
#     coherence_score = coherence_model_lda.get_coherence()
    
#     print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
#     if coherence_score > best_coherence:
#         best_coherence = coherence_score
#         best_topic = num_topics
# print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

#use the best model (result from above codes: 10 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=10,
                                                random_state=42)

#inspiration from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
topics_matrix = lda_model[corpus]
topics = []
# Iterate over each document's topic distribution
# Get the topic with the highest probability
for doc in topics_matrix:
    topic = max(doc, key=lambda x: x[1])[0]
    topics.append(topic)

merged_df['facts_topic'] = topics
merged_df = merged_df.reset_index(drop=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [fact, widow, tan, geok, tee, deceased, sue, c...
1       [fact, surrounding, circumstance, including, a...
2       [background, appellant, french, company, secon...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, suffice, exh, said, st...
                              ...                        
6788    [fact, accused, low, sze, song, low, year, old...
6789    [fact, giving, opinion, representation, amount...
6790    [fact, party, karan, bagga, litigant, proceedi...
6791                                                   []
6792    [fact, party, towa, company, incorporated, jap...
Name: processed_facts, Length: 6793, dtype: object
Finished preprocessing text
Performing topic modelling


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.121246 -0.009162       1        1  18.394148
0      0.088902  0.010847       2        1  15.537684
3      0.030430  0.042217       3        1  11.437624
4     -0.108537  0.001385       4        1   9.971369
6     -0.104829 -0.034864       5        1   8.759105
9      0.066696 -0.031323       6        1   8.355350
8      0.025374 -0.054561       7        1   7.776687
1     -0.103967 -0.018908       8        1   7.550549
5     -0.028842  0.104119       9        1   7.502391
7      0.013524 -0.009750      10        1   4.715094, topic_info=           Term          Freq         Total Category  logprob  loglift
812     accused  22664.000000  22664.000000  Default  30.0000  30.0000
9961       aeic  11453.000000  11453.000000  Default  29.0000  29.0000
3428     victim   8371.000000   8371.000000  Default  28.0000  28.0000
3487       drug   6243.000000   6243.000000  Default  27.0000  27.0000
823      charge  11253.000000  11253.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
214     however    505.014219   8088.481506  Topic10  -5.9039   0.2808
862       march    508.705715   8404.931487  Topic10  -5.8966   0.2497
485   affidavit    555.617633  14329.179730  Topic10  -5.8084  -0.1956
41        party    531.273519  15594.821341  Topic10  -5.8532  -0.3250
108     company    503.847018  16529.633381  Topic10  -5.9062  -0.4362

[898 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
31084      7  0.948557        aa
19160      5  0.938317       aad
10826      1  0.013903        ab
10826      2  0.903705        ab
10826      3  0.006952        ab
...      ...       ...       ...
6471       6  0.809821       ylp
6471      10  0.023138       ylp
18271      9  0.825404  youthful
38512      4  0.096289  zainudin
38512      5  0.890671  zainudin

[4669 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 5, 7, 10, 9, 2, 6, 8])

In [85]:
print(merged_df.isna().sum())

casename           0
area_of_law        0
Coram              0
court_level        0
issues             0
target             0
processed_facts    0
facts_topic        0
dtype: int64


#### Topic Modelling For Issues

In [86]:
data = []

merged_df['processed_issues'] = merged_df['issues'].apply(preprocess_text)
merged_df.drop(columns=['issues'], inplace=True)
print(merged_df["processed_issues"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for issue in merged_df['processed_issues']:
    for word in issue:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# print(lda_model.show_topics())

# best_coherence = -1
# best_lda = None
# for num_topics in range(5, 31, 5):
#     # Train LDA model
#     lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                                 id2word=dictionary,
#                                                 num_topics=num_topics,
#                                                 random_state=42)
#     # Compute coherence score
#     coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
#     coherence_score = coherence_model_lda.get_coherence()
    
#     print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
#     if coherence_score > best_coherence:
#         best_coherence = coherence_score
#         best_topic = num_topics
# print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

#use the best model (result from above codes: 25 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=25,
                                                random_state=42)

#inspiration from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
topics_matrix = lda_model[corpus]
topics = []
# Iterate over each document's topic distribution
# Get the topic with the highest probability
for doc in topics_matrix:
    topic = max(doc, key=lambda x: x[1])[0]
    topics.append(topic)
    
merged_df['issues_topic'] = topics
merged_df = merged_df.reset_index(drop=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [claim, dismissed, cost, high, decision, fook,...
1       [claim, decision, appeal, brought, background,...
2       [appeal, question, arise, appeal, follows, app...
3       [appeal, assistant, registrar, ground, judgmen...
4       [appeal, january, dismissed, give, reason, evi...
                              ...                        
6788    [sub, arise, consideration, whether, low, siva...
6789    [claim, conspiracy, defraud, fault, ken, sally...
6790    [relating, defence, justification, qualified, ...
6791                                                   []
6792    [background, dispute, towa, commenced, suit, a...
Name: processed_issues, Length: 6793, dtype: object
Finished preprocessing text
Performing topic modelling


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
24    -0.041158 -0.094579       1        1  7.215041
13    -0.055731  0.071590       2        1  6.996435
4     -0.043011 -0.041623       3        1  6.364334
12     0.069690 -0.080475       4        1  6.327884
14    -0.045392  0.054827       5        1  5.521054
11    -0.048650 -0.032361       6        1  4.779389
8     -0.024687 -0.034667       7        1  4.757361
7     -0.031010 -0.020072       8        1  4.375825
23     0.134487  0.003500       9        1  4.337060
2      0.104665  0.039677      10        1  3.849752
0     -0.046651 -0.021388      11        1  3.841561
5      0.104014  0.005775      12        1  3.836293
9     -0.027181  0.036384      13        1  3.683766
22     0.035142  0.018419      14        1  3.385522
20     0.019315  0.074827      15        1  3.365636
21    -0.053418  0.037354      16        1  3.191959
3      0.051615  0.024589      17        1  3.139716
1     -0.007792  0.009431      18        1  2.999724
18    -0.050322  0.024021      19        1  2.867505
10    -0.021188 -0.020580      20        1  2.839411
16     0.018832  0.005797      21        1  2.803414
6     -0.035622  0.025129      22        1  2.638818
19    -0.049910 -0.012475      23        1  2.303581
17     0.056902 -0.045596      24        1  2.292114
15    -0.012941 -0.027506      25        1  2.286846, topic_info=             Term          Freq          Total Category  logprob  loglift
1151      accused  39499.000000   39499.000000  Default  30.0000  30.0000
2962  arbitration  28290.000000   28290.000000  Default  29.0000  29.0000
2263      offence  29046.000000   29046.000000  Default  28.0000  28.0000
210       company  72165.000000   72165.000000  Default  27.0000  27.0000
801      property  44800.000000   44800.000000  Default  26.0000  26.0000
...           ...           ...            ...      ...      ...      ...
641           ltd   1543.064771  125376.475655  Topic25  -5.7301  -0.6196
575      judgment   1359.139507   47514.292906  Topic25  -5.8571   0.2238
823      question   1304.780978   42828.285987  Topic25  -5.8979   0.2868
8918          oct   1305.436528   54698.280353  Topic25  -5.8974   0.0427
748         party   1303.416982  109373.016332  Topic25  -5.8989  -0.6518

[2185 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
24874      5  0.170978      aa
24874      8  0.014868      aa
24874     10  0.029735      aa
24874     13  0.011151      aa
24874     16  0.003717      aa
...      ...       ...     ...
20726     24  0.013113  zoning
20726     25  0.668766  zoning
23715     10  0.959345     zul
23715     15  0.008721     zul
23715     17  0.017443     zul

[21421 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[25, 14, 5, 13, 15, 12, 9, 8, 24, 3, 1, 6, 10, 23, 21, 22, 4, 2, 19, 11, 17, 7, 20, 18, 16])

### Feature Engineering

In [87]:
print(merged_df)

              casename                                        area_of_law  \
0      2000_SGCA_1.pdf  {'civil procedure': ['pleadings'], 'res judica...   
1     2000_SGCA_10.pdf  {'contract': ['formation'], 'equity': ['defenc...   
2     2000_SGCA_11.pdf  {'contract': ['discharge'], 'damages': ['asses...   
3     2000_SGCA_12.pdf  {'courts and jurisdiction': ['court of appeal'...   
4     2000_SGCA_13.pdf                     {'criminal law': ['offences']}   
...                ...                                                ...   
6788  2023_SGHC_95.pdf           {'criminal law': ['statutory offences']}   
6789  2023_SGHC_96.pdf      {'tort': ['conspiracy', 'misrepresentation']}   
6790  2023_SGHC_97.pdf                 {'civil procedure': ['witnesses']}   
6791  2023_SGHC_98.pdf     {'insolvency law': ['schemes of arrangement']}   
6792  2023_SGHC_99.pdf            {'intellectual property': ['remedies']}   

                                          Coram court_level        target  

In [88]:
print(merged_df.isna().sum())

casename            0
area_of_law         0
Coram               0
court_level         0
target              0
processed_facts     0
facts_topic         0
processed_issues    0
issues_topic        0
dtype: int64


One-hot encoding

In [108]:
# one-hot encode aol
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(all_areas)

binary_aol_df = pd.DataFrame(binary_features, columns=mlb.classes_)
binary_aol_df = binary_aol_df.reset_index(drop=True)
processed_df = pd.concat([merged_df.drop('area_of_law', axis=1), binary_aol_df], axis=1)

print(processed_df.head(3))

# one-hot encode coram
processed_df = processed_df[processed_df['Coram'].apply(lambda x: isinstance(x, list))]
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(processed_df['Coram'])

binary_coram_df = pd.DataFrame(binary_features, columns=mlb.classes_)
binary_coram_df = binary_coram_df.reset_index(drop=True)
processed_df = pd.concat([processed_df.drop('Coram', axis=1), binary_coram_df], axis=1)

print(processed_df.head())

           casename                                      Coram court_level  \
0   2000_SGCA_1.pdf  [Chao Hick Tin; L P Thean; Yong Pung How]        SGCA   
1  2000_SGCA_10.pdf   [Chao Hick Tin; Tan Lee Meng; L P Thean]        SGCA   
2  2000_SGCA_11.pdf   [Chao Hick Tin; Tan Lee Meng; L P Thean]        SGCA   

       target                                    processed_facts  facts_topic  \
0  Favourable  [fact, widow, tan, geok, tee, deceased, sue, c...            8   
1  Favourable  [fact, surrounding, circumstance, including, a...            2   
2  No outcome  [background, appellant, french, company, secon...            0   

                                    processed_issues  issues_topic  \
0  [claim, dismissed, cost, high, decision, fook,...            19   
1  [claim, decision, appeal, brought, background,...             7   
2  [appeal, question, arise, appeal, follows, app...             0   

   "a larger sum being repaid"  "abet"  ...  work injury compensation act  \
0   

### splitting

In [109]:
X = processed_df.drop(columns=['target'])
y = processed_df['target']

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, remaining_index in stratified_split.split(X, y):
    X_train, X_test_val = X.iloc[train_index], X.iloc[remaining_index]
    y_train, y_test_val = y.iloc[train_index], y.iloc[remaining_index]

#balanced dataset (target variable was imbalanced Favourable 5006 Unfavourable 2523 No outcome 984)
#randomly found one online, can be changed -> need to check am i doing this right 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

#split further from X_test_val into X_val and X_test
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42, stratify=y_test_val)

In [110]:
## pipeline
X_train_resampled['SGCA'] = X_train_resampled['court_level'].apply(lambda x: 1 if x == 'SGCA' else 0)
X_train_resampled['SGHC'] = X_train_resampled['court_level'].apply(lambda x: 1 if x == 'SGHC' else 0)

X_test['SGCA'] = X_test['court_level'].apply(lambda x: 1 if x == 'SGCA' else 0)
X_test['SGHC'] = X_test['court_level'].apply(lambda x: 1 if x == 'SGHC' else 0)

X_val['SGCA'] = X_val['court_level'].apply(lambda x: 1 if x == 'SGCA' else 0)
X_val['SGHC'] = X_val['court_level'].apply(lambda x: 1 if x == 'SGHC' else 0)

X_train_resampled = X_train_resampled.drop(columns=['court_level', 'processed_facts', 'processed_issues'])
X_test = X_test.drop(columns=['court_level', 'processed_facts', 'processed_issues'])
X_val = X_val.drop(columns=['court_level', 'processed_facts', 'processed_issues'])

mapping = {'Favourable': 1, 'Unfavourable': 0, 'No outcome':0.5}
y_train_resampled, y_test, y_val = y_train_resampled.copy().map(mapping), y_test.copy().map(mapping), y_val.copy().map(mapping)

C:\Users\benhz\AppData\Local\Temp\ipykernel_34668\3775703400.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train_resampled['SGCA'] = X_train_resampled['court_level'].apply(lambda x: 1 if x == 'SGCA' else 0)
C:\Users\benhz\AppData\Local\Temp\ipykernel_34668\3775703400.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train_resampled['SGHC'] = X_train_resampled['court_level'].apply(lambda x: 1 if x == 'SGHC' else 0)


## Modeling

### CNN

In [112]:
# Perform modelling
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import tensorflow as tf

class Args:
  epochs = 20
  lr = 0.001
  use_cuda=False
  gamma = 0.7
  log_interval = 10
  seed = 3

args = Args()

device = torch.device("cuda" if args.use_cuda else "cpu")

In [113]:
X_train_resampled = X_train_resampled.iloc[:, 1:1596].copy()
X_train_resampled = torch.tensor(X_train_resampled.values, dtype=torch.float32).to(device)

In [114]:
print(f'Shape of X_train_resampled: {X_train_resampled.shape}')

Shape of X_train_resampled: torch.Size([8277, 1595])


In [115]:
X_test = X_test.iloc[:, 1:1596].copy()
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
print(f'Shape of X_test: {X_test.shape}')

X_val = X_val.iloc[:, 1:1596].copy()
X_val = torch.tensor(X_val.values, dtype=torch.float32).to(device)
print(f'Shape of X_val: {X_val.shape}')

Shape of X_test: torch.Size([1019, 1595])
Shape of X_val: torch.Size([1019, 1595])


In [116]:
y_train_resampled, y_test, y_val = torch.tensor(y_train_resampled.values).to(device), torch.tensor(y_test.values).to(device), torch.tensor(y_val.values).to(device)

In [117]:
class Net(nn.Module):  # Defines a new neural network architecture as a class that inherits from the PyTorch base class nn.Module.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, 3, 1,1, bias=True)
        # Define the first 1D convolution layer. Takes 1 input channel, outputs 32 channels, kernel size is 3, stride is 1, padding is 1.
        self.Bn1 = nn.BatchNorm1d(64)
        # Apply Batch Normalization to the output of the first convolutional layer.
        self.dropout = nn.Dropout(0.3)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        # Apply 1D Average Pooling after the first Batch Normalization. The kernel size and stride are 2.

        self.conv2 = nn.Conv1d(64, 64, 3, 1,1, bias=True)
        self.Bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(25472, 3, bias=True)
        # Define a linear layer (fully connected layer). It takes 32*12 inputs and outputs 5 nodes.


    def forward(self, x):
        x = F.relu(self.Bn1(self.conv1(x)))
        # Pass the input through the first convolutional layer, then Batch Normalization, and then apply ReLU activation.
        x = self.dropout(x)
        x = self.pool1(x)
        # Apply Average Pooling to the output of the previous step.
        x = F.relu(self.Bn2(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        # Flatten the output from the previous step. This is necessary because fully connected layers expect a 1D input.
        x = self.fc1(x)
        # Pass the flattened output through the fully connected layer. This is the output of the network.
        return x
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()  # Set the model to training mode

    for batch_idx, (data, target) in enumerate(train_loader):  # Loop over each batch from the training set
        data, target = data.to(device), target.to(device)  # Move the data to the device that is used

        target = target.long()  # Make sure that target data is long type (necessary for loss function)

        optimizer.zero_grad()  # Clear gradients from the previous training step
        output = model(data)  # Run forward pass (model predictions)

        loss = F.cross_entropy(output, target)  # Calculate the loss between the output and target
        loss.backward()  # Perform backpropagation (calculate gradients of loss w.r.t. parameters)
        optimizer.step()  # Update the model parameters

        if batch_idx % args.log_interval == 0:  # Print log info for specified interval
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0

    with torch.no_grad():  # Deactivates autograd, reduces memory usage and speeds up computations
        for data, target in test_loader:  # Loop over each batch from the testing set
            
            data, target = data.to(device), target.to(device)  # Move the data to the device that is used

            target = target.long()  # Convert target to long after adjusting value
            output = model(data)  # Run forward pass (model predictions)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # Sum up the batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability as the predicted output
            correct += pred.eq(target.view_as(pred)).sum().item()  # Count correct predictions

    test_loss /= len(test_loader.dataset)  # Calculate the average loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
    return correct  # Return the number of correctly classified samples


In [118]:
print(X_train_resampled.shape)
torch.manual_seed(args.seed)

model = Net().to(device)

for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())

#Form training and testing dataset
optimizer = optim.Adam(model.parameters(), lr=args.lr)

train_dataset = torch.utils.data.TensorDataset(X_train_resampled, y_train_resampled)
test_dataset = torch.utils.data.TensorDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

#Model training
ACC = 0
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    ACC_ = test(model, device, test_loader)
    if ACC_>ACC or ACC_ == ACC:
        ACC = ACC_
        torch.save(model.state_dict(), "Baseline_CNN.pt")

    scheduler.step()

print(ACC)


torch.Size([8277, 1595])
conv1.weight 	 torch.Size([64, 1, 3])
conv1.bias 	 torch.Size([64])
Bn1.weight 	 torch.Size([64])
Bn1.bias 	 torch.Size([64])
Bn1.running_mean 	 torch.Size([64])
Bn1.running_var 	 torch.Size([64])
Bn1.num_batches_tracked 	 torch.Size([])
conv2.weight 	 torch.Size([64, 64, 3])
conv2.bias 	 torch.Size([64])
Bn2.weight 	 torch.Size([64])
Bn2.bias 	 torch.Size([64])
Bn2.running_mean 	 torch.Size([64])
Bn2.running_var 	 torch.Size([64])
Bn2.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([3, 25472])
fc1.bias 	 torch.Size([3])


RuntimeError: Given groups=1, weight of size [64, 1, 3], expected input[1, 64, 1595] to have 1 channels, but got 64 channels instead

### Random Forest

## Evaluation

In [ ]:
## Perform Evaluation